# Lecture 13: Practical analyses in Python

## Dictionaries
Python defines a powerful type called a dictionary or `dict`.

### Mutable versus immutable types
Before using dictionaries, we need to understand the difference between [mutable and immutable types](https://medium.com/@meghamohan/mutable-and-immutable-side-of-python-c2145cf72747).

Let's review the types we've learned so far:
#### Integers

In [1]:
x_int = 1

type(x_int)

int

An integer is immutable, we can't change it's value by reassigment:

In [2]:
y_int = x_int
y_int += 1
print(f"y_int = {y_int}, x = {x_int}")

y_int = 2, x = 1


#### Floats

In [3]:
x_float = 3.7

type(x_float)

float

Floats are also immutable:

In [4]:
y_float = x_float
y_float += 1
print(f"y_float = {y_float}, x = {x_float}")

y_float = 4.7, x = 3.7


#### Lists

In [5]:
x_list = [1, 2, 3]

type(x_list)

list

But lists are mutable:

In [6]:
y_list = x_list
y_list.append(4)

print(f"y_list = {y_list}, x_list = {x_list}")

y_list = [1, 2, 3, 4], x_list = [1, 2, 3, 4]


Notice how the value of `x_list` has changed too! Lists are mutable, meaning they point to the mutable object, not to its value.
Be careful with mutable objects!

There is an immutable object that has some similar properties to a list called a `tuple`, but we aren't going to go into those right now.

#### Strings

In [7]:
x_str = 'hello'

type(x_str)

str

Strings are immutable:

In [8]:
y_str = x_str
y_str += ' friend'

print(f"y_str = {y_str}, x_str = {x_str}")

y_str = hello friend, x_str = hello


### What is a dictionary?
Dictionaries are "look up tables" that can be used to map keys to values:

In [9]:
teacher_dict = {'best_teacher': 'Jesse',
                'worst_teacher': 'Rasi'}

teacher_dict

{'best_teacher': 'Jesse', 'worst_teacher': 'Rasi'}

This dictionary maps the best and worst teachers to their names.
We can look up the value for a key:

In [10]:
teacher_dict['best_teacher']

'Jesse'

Note that we get an error if they key doesn't exist:

In [11]:
teacher_dict['second_best_teacher']

KeyError: 'second_best_teacher'

We can get all of the keys:

In [19]:
teacher_dict.keys()

dict_keys(['best_teacher', 'worst_teacher'])

Or the values:

In [16]:
teacher_dict.values()

dict_values(['Jesse', 'Rasi'])

Or the items, which are tuples with keys and values:

In [14]:
teacher_dict.items()

dict_items([('best_teacher', 'Jesse'), ('worst_teacher', 'Rasi')])

We can add entries to a dictionary like this:

In [20]:
teacher_dict['most_fashionable_teacher'] = 'Trevor'

teacher_dict

{'best_teacher': 'Jesse',
 'worst_teacher': 'Rasi',
 'most_fashionable_teacher': 'Trevor'}

Note that the above cell shows that dictionaries are mutable, as we changed the dictionary by adding to it.

An advantage of a dictionary is that it is very fast to look up key / value pairs even if there are lots of entries in the dictionary.

So why did I make a big deal about mutable versus immutable?
Dictionary keys can **only** be immutable. 
So keying a dictionary with a mutable type (such as a `list`) is not allowed:

In [21]:
teacher_dict[['previous_teacher', 'current_teacher']] = [['Erick', 'Jesse']]

TypeError: unhashable type: 'list'

But dictionary values can be mutable:

In [22]:
teacher_dict['previous_and_current_teachers'] = [['Erick', 'Jesse']]

teacher_dict

{'best_teacher': 'Jesse',
 'worst_teacher': 'Rasi',
 'most_fashionable_teacher': 'Trevor',
 'previous_and_current_teachers': [['Erick', 'Jesse']]}

## Using a dictionary in a function
Now we will use a dictionary to implement a function that gets the reverse complement of a sequence.

Here is the desired signature of the function. Can you write this function?

In [41]:
def reverse_complement(seq):
    """Get reverse complement of a DNA sequence.
    
    Parameters
    -----------
    seq : str
        Uppercase DNA sequence.
        
    Returns
    -------
    str
        Reverse complement of the sequence in upper case.
        
    Example
    --------
    >>> reverse_complement('ATGCAC')
    'GTGCAT'
    
    """
    rc_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    revcomplement = []
    for nt in reversed(seq.upper()):
        revcomplement.append(rc_dict[nt])
    return ''.join(revcomplement)

Once the function is written, you can use it:

In [35]:
reverse_complement('ATGCAC')

'GTGCAT'

In [42]:
reverse_complement('aTGCTAAAAGTTCAGGATACAGGTAAN')

'NTTACCTGTATCCTGAACTTTTAGCAT'

## Regular expressions
The `re` module is for "regular expressions". 
These are very useful for parsing strings.

Here is a common example from my work with influenza.
You download some strains from the database, and they have names that look like this:

In [43]:
strain1 = 'A/New York/3/1994 (H3N2)'
strain2 = 'A/California/3/X/2003 (H12N1)'
strain3 = 'A/Perth/2009 (H3N2)'

strains = [strain1, strain2, strain3]

You want to get some information out of these, like the year.

Regular expressions allow you to define patterns in a string.
Here we build a regular expression that gets the subtype out and then use a dictionary to count how many sequences there are of each subtype:

In [57]:
import re

# regular expression that gets year and subtype
strainmatch = re.compile(
        '(?P<year>\d{4}) \((?P<subtype>H\d+N\d+)\)$')

subtype_counter = {}  # dict to store the results

for strain in strains:  # loop over all strains
    m = strainmatch.search(strain)
    year = m.group('year')
    subtype = m.group('subtype')
    if subtype in subtype_counter:
        subtype_counter[subtype] += 1
    else:
        subtype_counter[subtype] = 1
        
print(subtype_counter)

{'H3N2': 2, 'H12N1': 1}


There are lots of handy special codes in the Python regular expression module (see [here](https://docs.python.org/3.7/library/re.html)), and you can use them to do almost any type of string matching.

## Using regular expressions to parse barcodes
Now we will use regular expressions to parse barcodes from nucleotide sequences.
For instance, you might have to do this in a single-cell RNA-seq experiment where there is a barcode at the end of each read telling you the cell that the read came from.

Imagine that our valid molecules should have sequences like this:

`CTAGCNNNNNNGATCA`

See how there is a 6-nucleotide barcode in the center of the sequence.
We have a list of sequences, and want to parse through them to figure out which ones meet the expected pattern--and get the barcode from those that do:

In [60]:
seqs = ['CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CCAGCatagcaGATCA',  # does not have expected 5' sequence
        'CTAGCtacagGATCA',   # barcode too short
        'CTAGCgaccatGATCA',  # has barcode GACCAT
        'CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CTAGCatcgatGGTCA',  # does not have expected 3' sequence
        ]

Write a function that parses these barcoded sequences and gets the ones with valid barcodes.
In doing this, note that:

  1. If you have a string `s`, `s.upper()` makes it all uppercase.
  2. Consider using the following regular expression options (all described [here](https://docs.python.org/3.7/library/re.html):
    - `[]` indicates a set of characters.
    - `{n}` indicates number of occurrences.
    - `(?P<name>)` indicates a group with name `name`.
    - The `match` method finds a match at the beginning of the string.
    
Below I've written the function documentation, try to implement it:

In [66]:
def count_barcodes(seqs, bclen=6, upstream='CTAGC', downstream='GATCA'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs : list
        DNA sequences.
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
    downstream : str
        Sequence downstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    regex = re.compile(upstream +
                       f"(?P<bc>[ACGT]{{{bclen}}})" +
                       downstream
                       )
    counts = {}
    for seq in seqs:
        seq = seq.upper()
        m = regex.match(seq)
        if m:
            bc = m.group('bc')
            if bc in counts:
                counts[bc] += 1
            else:
                counts[bc] = 1
    return counts


Run the function once you've implemented it. Does it give the right result?

In [67]:
count_barcodes(seqs)

{'ATCGAT': 2, 'GACCAT': 1}

## Biopython
[Biopython](https://biopython.org/) is a package that has lots of useful functions for computational biology.

It is very handy for things like reading in sequences in many different formats: [Bio.SeqIO](https://biopython.org/wiki/SeqIO) is your friend!

(Do note that if you are analyzing truly large datasets, `Biopython` is not very fast and you may want to use something like [pysam](https://pysam.readthedocs.io/en/latest/api.html); but `Biopython` is a good starting point).

### Reading in a file
I have included the file [barcodes_R1.fastq](barcodes_R1.fastq), which has some FASTQ sequences in it.

First, let's just see what the beginning of that file looks like:

In [ ]:
! head -n 8 barcodes_R1.fastq

Now let's use `Biopython` to read the FASTQ entries.

First, import `Biopython.SeqIO`:

In [ ]:
import Bio.SeqIO

Now read in the sequencing reads:

In [ ]:
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

How many reads were there?

In [ ]:
print(f"Found {len(seqreads)} sequencing reads.")

Let's look at the first read:

In [ ]:
seqreads[0]

You can see that it has a lot of information, including the header, quality scores, etc.

For our purposes, we will just convert the sequence part into a string for each sequence:

In [ ]:
seqreads_str = [str(s.seq) for s in seqreads]

Make sure we still have the same number of sequencing reads, and look at the first one:

In [ ]:
assert len(seqreads_str) == len(seqreads)

seqreads_str[0]

## A real biological analysis: parsing barcodes
The reads that we just read as `seqreads_str` come from a real sequencing run of influenza virus HA and NA genes.

The sequences are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The sequencing run reads from the reverse end of the molecules, so the first thing in the sequencing reads is the barcode followed by the constant sequence and the end of HA or NA.

We want to determine which reads have valid sequences, get the barcodes out of strings, figure out if the barcode matches to HA or NA, and count the barcodes.
So this requires setting up an analysis that does the following:

 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 
Can you write code that does this?